# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame

In [2]:
city_weather_df = pd.read_csv('cities.csv')
city_weather_df

,City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness,Wind Speed (MPH),Country,Date
0,nikolskoye,59.7035,30.7861,10.99,79,75,13.42,RU,1612630830
1,chitipa,-9.7024,33.2697,68.04,91,99,2.53,MW,1612630831
2,placido de castro,-10.3353,-67.1856,77.76,82,99,2.98,BR,1612630831
3,naze,28.3667,129.4833,61.47,82,98,7.16,JP,1612630832
4,kota bahru,6.1333,102.2386,77.00,83,20,5.75,MY,1612630832
...,...,...,...,...,...,...,...,...,...
580,bubaque,11.2833,-15.8333,79.63,59,23,16.37,GW,1612631036
581,mattoon,39.4831,-88.3728,25.00,58,1,5.75,US,1612631036
582,talcahuano,-36.7167,-73.1167,60.80,82,90,4.61,CL,1612630789
583,bandarbeyla,9.4942,50.8122,76.93,78,15,13.29,SO,1612631037


### Humidity Heatmap
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = list(zip(city_weather_df['Latitude'], city_weather_df['Longitude']))
fig = gmaps.figure(map_type='SATELLITE')
heatmap_layer = gmaps.heatmap_layer(locations, weights=city_weather_df['Humidity (%)'],
                                    max_intensity=100,dissipating=True,
                                    point_radius=9, opacity=0.4)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
good_places_df = city_weather_df[city_weather_df['Temperature (F)'] < 110]
good_places_df = good_places_df[good_places_df['Temperature (F)'] > 85]
good_places_df = good_places_df[good_places_df['Humidity (%)'] < 70]
good_places_df = good_places_df[good_places_df['Wind Speed (MPH)'] < 6]
good_places_df = good_places_df[good_places_df['Cloudiness'] < 50]

good_places_df.reset_index(inplace=True, drop=True)
good_places_df

,City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness,Wind Speed (MPH),Country,Date
0,champerico,14.3000,-91.9167,86.00,39,0,4.61,GT,1612630852
1,oriximina,-1.7656,-55.8661,94.98,40,28,4.23,BR,1612630902
2,pontes e lacerda,-15.2261,-59.3353,90.52,43,28,3.91,BR,1612630848
3,san juan,-31.5375,-68.5364,92.10,21,0,5.59,AR,1612630954
4,chivilcoy,-34.8957,-60.0167,90.00,69,0,3.00,AR,1612631000


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
good_places_df['Hotel name'] = 'name'
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for i in range(len(good_places_df)):
    ll_dot = str(good_places_df.loc[i,'Latitude']) +',' + str(good_places_df.loc[i,'Longitude'])
    params = {"location": ll_dot,
#              "radius": 5000,
              "rankby": "distance",
              "type": "lodging",
              "key": g_key}
    
    response = requests.get(base_url, params=params)
    hotel_data = response.json()
    good_places_df.loc[i,'Hotel name'] = hotel_data['results'][0]['name']

good_places_df

,City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness,Wind Speed (MPH),Country,Date,Hotel name
0,champerico,14.3000,-91.9167,86.00,39,0,4.61,GT,1612630852,Hotel Miramar
1,oriximina,-1.7656,-55.8661,94.98,40,28,4.23,BR,1612630902,Pousada Santo Antonio
2,pontes e lacerda,-15.2261,-59.3353,90.52,43,28,3.91,BR,1612630848,Hotel Veneza
3,san juan,-31.5375,-68.5364,92.10,21,0,5.59,AR,1612630954,Gonzalez Roberto F
4,chivilcoy,-34.8957,-60.0167,90.00,69,0,3.00,AR,1612631000,Tu Pension


In [ ]:
# NOTE: Do not change any of the code in this cell
# Didn't understand this code to good, so decided to figure out another way to do it myself

In [7]:
locations = good_places_df[["Latitude", "Longitude"]]
# Add marker layer ontop of heat map
list_names = good_places_df['Hotel name']
fig.add_layer(gmaps.marker_layer(locations,hover_text=list_names))

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))